# *ml-1K-loop-with-n_jobs-and-premade.ipynb*

# Running models sequentially, but each model uses n_jobs=-1 for parallelization; use premade train/val datasets

Version 1K: Accomodates new model architectures. Reads in premade train and val datasets for each fold. (This makes the "Assembling datasets" portion much faster!)

This is a work-in-progress-notebook. Currently, it allows us to: specify a models bank for 160 Random Forest and Extra Trees models; assemble the datasets for different folds of year-wise cross-validation; run the models for 12 combinations of region, crop, compositing scheme, and in-season date; and record results (accuracies and uncertainties) in csv files.

Function definitions at the top of the notebook; then sequential model runs (but each RF/ET run using parallel n_jobs=-1).


In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
def create_models_bank(architecture):
    models_bank = {}
    increment = 0
    if architecture in ('RF','ET','RFET','ETRF'):
        for n_estimators in [200, 500]:
            for max_features in [0.05, 0.1, 0.2, 0.4, 1.0]:
                for min_samples_split in [2, 4]:
                    for bootstrap in [False, True]:
                        for class_weight in [None, 'balanced']:

                            increment += 1

                            three_digit = str(increment).zfill(3)
                            models_bank[three_digit] = {
                                'n_estimators': n_estimators,
                                'max_features': max_features,
                                'bootstrap': bootstrap,
                                'min_samples_split': min_samples_split,
                                'class_weight': class_weight,
                                'n_jobs': -1
                            }

    if architecture in ('LR'):
        # 50 regularized models
        for l1_ratio in [0, 0.1, 0.5, 0.9, 1]:
            for C in [1, 10**-2, 10**-4, 10**-6, 10**-8]:
                for class_weight in [None, 'balanced']:
                    increment += 1
                    three_digit = str(increment).zfill(3)
                    models_bank[three_digit] = {
                        'solver': 'saga',
                        'penalty': 'elasticnet',
                        'l1_ratio': l1_ratio,
                        'C': C,
                        'class_weight': class_weight,
                        'max_iter': 100000,
                        'random_state': 19
                    }
        # 2 nonregularized models
        for class_weight in [None, 'balanced']:
            increment += 1
            three_digit = str(increment).zfill(3)
            models_bank[three_digit] = {
                'solver': 'saga',
                'penalty': 'none',
                'class_weight': class_weight,
                'max_iter': 20000,
                'random_state': 19
            }

    return models_bank

In [ ]:
def grab_premade_X_y_train_val(training_sample_size,validation_sample_size,tile,
                     val_year,scheme_name,crop_of_interest_id,in_season):

    loc = f'../data/premade_{training_sample_size}_{validation_sample_size}'
    strings = []
    for arg in [tile,val_year,scheme_name,crop_of_interest_id,in_season]:
        strings.append(f'{arg}')
    most_of_name = '_'.join(strings) 
    
    Xy_trainval= ['X_train', 'X_val', 'y_train', 'y_val']
    
    d = {}
    for spec in Xy_trainval:
        d[spec] = np.load(f'{loc}/{most_of_name}_{spec}.npy')
    
    return d['X_train'], d['X_val'], d['y_train'], d['y_val']


In [ ]:
def fit_predict_report(model_name,
                      model,
                      training_sample_size,
                      validation_sample_size,
                      tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                      in_season,
                      from_premade=True
                      ):
    
    # produce csv_name
    exempt = ['years', 'model']
    param_value_strings = [f'{model_name}',
                      f'{training_sample_size}',
                      f'{validation_sample_size}',
                      f'{tile}',
                      f'{scheme_name}',
                      f'{crop_of_interest_id}',
                      f'{in_season}']
    csv_name = '_'.join(param_value_strings) +'.csv'

    # check whether previously run and, if so, end the effort
    if csv_name in os.listdir('../data/results/'):
        return 'If you see this, the specified model was previously run.'

    print(f'-- Process for {csv_name} --')
    
    # below is actually fitting and predicting and reporting
    
    conf = []

    for val_year in years:
        print('Starting a fold...')
        print('> Assembling the datasets')
        # NEED: X_train, y_train, X_val, y_val
        if from_premade==True: 
            X_train, X_val, y_train, y_val = grab_premade_X_y_train_val(training_sample_size,
                    validation_sample_size,tile,val_year,scheme_name,
                    crop_of_interest_id,in_season)
        
        if from_premade!=True:
            return 'This function in this notebook is only for from_premade=True'
        
        print('> Fitting the model on the training set')
        model.fit(X_train, y_train)
        print('> Predicting on the validation set')
        pred = model.predict(X_val)

        print('> Recording performance metrics')
        act = y_val
        ActPred_00 = sum((act==0) & (pred==0))
        ActPred_01 = sum((act==0) & (pred==1))
        ActPred_10 = sum((act==1) & (pred==0))
        ActPred_11 = sum((act==1) & (pred==1))
        conf_1yr = [ActPred_00, ActPred_01, ActPred_10, ActPred_11]

        conf.append(conf_1yr)
        print('Finished a fold.')

    carr = np.array(conf)

    carr = np.row_stack([carr,np.full((2,4),-1)])

    # above we added the totals row
    # now we need to add the columns for precision and recall

    # create dataframe
    cdf = pd.DataFrame(data = carr,
                      index = [f'ValYear{yr}' for yr in years]+['Mean','StdE'],
                      columns = ['ActPred_00', 'ActPred_01', 
                                 'ActPred_10', 'ActPred_11']
                      )

    cdf['Precision'] = cdf.ActPred_11 / (cdf.ActPred_01 + cdf.ActPred_11)
    cdf['Recall'] = cdf.ActPred_11 / (cdf.ActPred_10 + cdf.ActPred_11)
    cdf['F1'] = 2*cdf.Precision*cdf.Recall / (cdf.Precision + cdf.Recall)
    for col in ['Precision','Recall','F1']:
        cdf.at['Mean',col] = np.mean(cdf.loc[:'ValYear2022',col])
        cdf.at['StdE',col] = np.std(cdf.loc[:'ValYear2022',col])
    
    
    param_strings = [f'# model_name: {model_name}',
                     f'# model: {model}',
                      f'# training_sample_size: {training_sample_size}',
                      f'# validation_sample_size: {validation_sample_size}',
                      f'# tile: {tile}',
                      f'# scheme_name: {scheme_name}',
                      f'# crop_of_interest_id: {crop_of_interest_id}',
                      f'# in_season: {in_season}']
    comment = '\n'.join(param_strings) + '\n' 
    with open(f'../data/results/{csv_name}', 'a') as f:
        f.write(comment)
        cdf.to_csv(f)
    
    print(f'Find results in ../data/results/{csv_name}')
    
    return f'Find results in ../data/results/{csv_name}'

## Run models sequentially


In [ ]:
models_bank = create_models_bank('LR')
training_sample_size = 0.001
validation_sample_size = 0.001

for tile_coiid in [('10SFH',75),('15TVG',1)]:
    for scheme_name in ['14day','5day']:
        for in_season in [160, 230, None]:
            for three_digit in models_bank.keys():

                ## Parameters dictionary p LOGISTIC REGRESSION
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'LR' + three_digit,
                'model': make_pipeline(StandardScaler(),
                                   LogisticRegression(**models_bank[three_digit])),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                #fit_predict_report(**p) # run with the above parameters
                fit_predict_report(**p)

### Testing ground for Logistic Regression

In [ ]:
#models_bank = create_models_bank('LR')
training_sample_size = 0.001
validation_sample_size = 0.001

for tile_coiid in [('10SFH',75),('15TVG',1)]:
    for scheme_name in ['14day','5day']:
        for in_season in [160, 230, None]:
            #for three_digit in models_bank.keys():

            bank_specs = { 'solver': 'saga',
                        'penalty': 'elasticnet',
                        'l1_ratio': 0.5,
                        'C': 0.1,
                        'class_weight': 'balanced',
                        'max_iter': 100000,
                        'random_state': 19
                         }

            ## Parameters dictionary p RANDOM FOREST
            p = {

            ## SPECIFY MODEL ##
            'model_name': 'LRtestK_saga',
            'model': make_pipeline(StandardScaler(),
                                   LogisticRegression(**models_bank[three_digit])),
            'training_sample_size': training_sample_size,
            'validation_sample_size': validation_sample_size,

            ## SPECIFY TILE AND SCHEME ##
            'tile': tile_coiid[0],
            'years': [2018, 2019, 2020, 2021, 2022],
            'scheme_name': scheme_name,
            'crop_of_interest_id': tile_coiid[1], 
            'in_season': in_season
            }

            #fit_predict_report(**p) # run with the above parameters
            fit_predict_report(**p)